In [ ]:
Simport pandas as pd

In [ ]:
test_df = pd.read_csv('/content/drive/MyDrive/Summarizer_AI/Dataset/Fixed/test_df.csv')

In [ ]:
test_df.head()

,Unnamed: 0,original_text,abstractive_summary
0,6475,liputan6 com jakarta kejatuhan abdurrahman ...,bekas presiden abdurrahman wahid tumbang lanta...
1,9607,liputan6 com jakarta mantan presiden abdurr...,mantan presiden abdurrahman wahid mengatakan s...
2,9470,liputan6 com jakarta bank muamalat bergemin...,bank muamalat akan meningkatkan jumlah kreditn...
3,360,liputan6 com jakarta secara keseluruhan law...,secara keseluruhan lawatan megawati ke as dan ...
4,4329,liputan6 com jakarta sejumlah prajurit tni ...,prajurit tni diminta meningkatkan sikap profes...


In [ ]:
columns_to_delete = ['Unnamed: 0']

test_df = test_df.drop(columns=columns_to_delete)
test_df.head()

,original_text,abstractive_summary
0,liputan6 com jakarta kejatuhan abdurrahman ...,bekas presiden abdurrahman wahid tumbang lanta...
1,liputan6 com jakarta mantan presiden abdurr...,mantan presiden abdurrahman wahid mengatakan s...
2,liputan6 com jakarta bank muamalat bergemin...,bank muamalat akan meningkatkan jumlah kreditn...
3,liputan6 com jakarta secara keseluruhan law...,secara keseluruhan lawatan megawati ke as dan ...
4,liputan6 com jakarta sejumlah prajurit tni ...,prajurit tni diminta meningkatkan sikap profes...


In [ ]:
!pip install datasets transformers torch accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import torch
from datasets import load_dataset, Dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import DataCollatorForSeq2Seq, pipeline, EncoderDecoderModel

In [ ]:
# Check if GPU is available and set device accordingly
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')

Using device: cuda


In [ ]:
from tqdm import tqdm

def generate_abstractive_summaries_for_df(df):
    model_path = "migz117/Abstractive_T5"
    model = AutoModelForSeq2SeqLM.from_pretrained(model_path).to(device)
    tokenizer = AutoTokenizer.from_pretrained(model_path)

    abstractive_summaries = []

    for text in tqdm(df['original_text'], desc="Generating Summaries"):
        try:
            inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)
            summary_ids = model.generate(**inputs, max_length=200)
            abstractive_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
            abstractive_summaries.append(abstractive_summary.strip())

        except Exception as e:
            print(f"Error generating summary for text: {text[:50]}...")
            print(f"Error: {e}")
            abstractive_summaries.append("")

    return pd.DataFrame({
        'abstractive_summary': abstractive_summaries
    })

abstractive_summary_T5_df = generate_abstractive_summaries_for_df(test_df)
abstractive_summary_T5_df

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/801 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/20.6k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/793k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.59k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
Generating Summaries: 100%|██████████| 3738/3738 [46:19<00:00,  1.35it/s]


,abstractive_summary
0,pengamat politik dewi fortuna anwar menyebutka...
1,gus dur mengatakan beberapa menteri masih terl...
2,bank muamalat berencana meningkatkan kreditnya...
3,secara keseluruhan lawatan presiden megawati s...
4,para prajurit tni mengikuti gladi bersih perin...
...,...
3733,pln akan menerapkan tarif listrik regional di ...
3734,mantan presiden soeharto merayakan ulang tahun...
3735,puluhan aktivis perempuan dari berbagai pergur...
3736,ratusan kepala keluarga pengungsi kembali ke k...


In [ ]:
from google.colab import files

csv_file_path = 'abstractive_summary_T5_df.csv'
abstractive_summary_T5_df.to_csv(csv_file_path, index=False)

files.download(csv_file_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
abstractive_summary_T5_df = pd.read_csv('/content/abstractive_summary_T5_df.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Summarizer_AI/Dataset/Fixed/test_df.csv')

In [ ]:
abstractive_summary_T5_df.head()

,abstractive_summary
0,pengamat politik dewi fortuna anwar menyebutka...
1,gus dur mengatakan beberapa menteri masih terl...
2,bank muamalat berencana meningkatkan kreditnya...
3,secara keseluruhan lawatan presiden megawati s...
4,para prajurit tni mengikuti gladi bersih perin...


In [ ]:
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=f94b5e725514d24002347bffadbe161697461943733841658d2cdf5705601f79
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


# Hasil FineTuning

In [ ]:
from rouge_score import rouge_scorer

# Siapkan referensi dan kandidat
references = test_df['abstractive_summary'].tolist()
candidates = abstractive_summary_T5_df['abstractive_summary'].tolist()

# Inisialisasi Rouge Scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2'], use_stemmer=True)

# Hitung skor ROUGE
rouge1_precisions = []
rouge1_recalls = []
rouge1_f1s = []
rouge2_precisions = []
rouge2_recalls = []
rouge2_f1s = []

# Iterasi untuk setiap pasang kandidat dan referensi
for candidate, reference in zip(candidates, references):
    scores = scorer.score(reference, candidate)

    # ROUGE-1
    rouge1_precisions.append(scores['rouge1'].precision)
    rouge1_recalls.append(scores['rouge1'].recall)
    rouge1_f1s.append(scores['rouge1'].fmeasure)

    # ROUGE-2
    rouge2_precisions.append(scores['rouge2'].precision)
    rouge2_recalls.append(scores['rouge2'].recall)
    rouge2_f1s.append(scores['rouge2'].fmeasure)

# Cetak hasil agregasi
print(f"ROUGE-1 Precision: {sum(rouge1_precisions) / len(rouge1_precisions):.4f}")
print(f"ROUGE-1 Recall: {sum(rouge1_recalls) / len(rouge1_recalls):.4f}")
print(f"ROUGE-1 F1: {sum(rouge1_f1s) / len(rouge1_f1s):.4f}")

print(f"\nROUGE-2 Precision: {sum(rouge2_precisions) / len(rouge2_precisions):.4f}")
print(f"ROUGE-2 Recall: {sum(rouge2_recalls) / len(rouge2_recalls):.4f}")
print(f"ROUGE-2 F1: {sum(rouge2_f1s) / len(rouge2_f1s):.4f}")

ROUGE-1 Precision: 0.3909
ROUGE-1 Recall: 0.4258
ROUGE-1 F1: 0.4019

ROUGE-2 Precision: 0.2162
ROUGE-2 Recall: 0.2341
ROUGE-2 F1: 0.2214


In [ ]:
!pip install bert-score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.2 MB/s eta 0:00:00


In [ ]:
import bert_score

references = test_df['abstractive_summary'].tolist()
candidates = abstractive_summary_T5_df['abstractive_summary'].tolist()

P, R, F1 = bert_score.score(candidates, references, lang="id", model_type="bert-base-uncased")

# Aggregate scores (mean)
print(f"BERTScore fine-tuned T5 model abstractive summary:")
print(f"BERTScore Precision: {P.mean().item():.4f}")
print(f"BERTScore Recall: {R.mean().item():.4f}")
print(f"BERTScore F1: {F1.mean().item():.4f}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BERTScore fine-tuned T5 model abstractive summary:
BERTScore Precision: 0.7285
BERTScore Recall: 0.7414
BERTScore F1: 0.7343
